<a href="https://colab.research.google.com/github/small-qiu/Deep_learning/blob/master/PDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 深度学习求解高微分方程

![image.png](attachment:image.png)

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
     raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
2.3.0


In [3]:
import time  
import math
import tensorflow.compat.v1 as tf
import numpy as np
from tensorflow.python.training.moving_averages import assign_moving_average   # 移动平均
from scipy.stats import multivariate_normal as normal            # 产生正态分布随机数
from tensorflow.python.ops import control_flow_ops               #用于控制流  
from tensorflow import random_normal_initializer as norm_init    #生成具有正态分布的张量的初始化器
from tensorflow import random_uniform_initializer as unif_init   #生成具有均匀分布的张量的初始化器
from tensorflow import constant_initializer as const_init        #生成具有常量值的张量的初始化器

tf.disable_v2_behavior()

class SolveAllenCahn (object):
    """ The fully-connected neural network model .全连接"""  
    def __init__ (self,sess):
        self.sess = sess     # 会话
        # parameters for the PDE
        self.d = 100      # 数据的维度
        self.T = 0.3      # 每一条路径的时间长度
        # parameters for the algorithm
        self.n_time = 20     # 有20个网络构成
        self.n_layer = 4     # 神经网络的层数
        self.n_neuron = [self.d ,self.d +10 ,self.d +10, self.d ]    # 各层神经元的个数，对应输入、隐藏层1、隐藏层2、输出
        self.batch_size = 64      # 一次用到个路径计算，64*4=256
        self.valid_size = 256     # 256个蒙特卡洛样本（路径）
        self.n_maxstep = 4000     #迭代步数
        self.n_displaystep = 1000     # 每100步print一次
        self.learning_rate = 5e-4    # 学习率
        self.Yini = [0.3,0.6]       # 初始值Y0的最大最小值
        # some basic constants and variables
        self.h = (self.T +0.0)/self.n_time    # 每一个路径中数据时间间隔，delta t
        self.sqrth = math.sqrt(self.h)    # 根号delta t，后面用于计算
        self.t_stamp = np.arange(0,self.n_time)*self.h  # 时间戳，累计的时间
        self._extra_train_ops = []  # batch移动平均值操作，其中需要额外训练的beta和gamma

    def train(self):
        # 主要函数，用于神经网络的训练
        start_time = time.time()   # 起始时间
        # train operations创建新的tensorflow变量,name='global_step'，用产量生成器
        self.global_step = \
            tf.get_variable('global_step', [] ,
                              initializer = tf.constant_initializer(1),
                              trainable = False,dtype = tf.int32 )   # 没有添加到要训练的变量列表，计步器
        trainable_vars = tf.trainable_variables()  # 查看可训练的变量
        grads = tf.gradients(self.loss,trainable_vars)  # loss可训练变量的梯度
        optimizer = tf.train.AdamOptimizer(self.learning_rate)    # 梯度优化器
        apply_op = \
            optimizer.apply_gradients(zip(grads,trainable_vars) ,    # 将梯度用来更新trainable_vars列表中的东西
                                          global_step = self.global_step)   # 更新梯度和迭代次数
        
        train_ops = [apply_op] + self._extra_train_ops   # 添加操作，相当于list1.extand(list2)
        self.train_op = tf.group(* train_ops)   # tf.group(*train_ops)组合*train_ops的操作
        
        self.loss_history = []   # 用于记录loss值
        self.init_history = []   # 用于记录Y0的值
        
        # for validation,256条蒙特卡洛做验证集
        dW_valid , X_valid = self.sample_path(self.valid_size)   # 生成数据
        feed_dict_valid = { self.dW : dW_valid,   # 喂数据给buildmodel中的占位符
                            self.X : X_valid,
                            self.is_training: False }   # 不列入迭代范围
        # initialization
        step = 1
        self.sess.run (tf.global_variables_initializer())  # 初始化全局变量
        
        # 运行框架
        temp_loss = self.sess.run(self.loss ,
                                  feed_dict = feed_dict_valid )  # 计算损失
        
        temp_init = self.Y0.eval()[0] # # 取出值，Y0是二维张量
        self.loss_history.append(temp_loss)  # 记录loss
        self.init_history.append(temp_init)  # 记录Y0
        print("step : %5u , loss : %.4e , " % \
                (0 ,temp_loss ) + \
                "Y0 : % .4e , runtime : %4u " % \
                (temp_init, time.time()-start_time + self.t_bd))   # 打印step=0的状态
        
        # begin sgd iteration，0-4000步
        for _ in range (self.n_maxstep +1):   
            step = self.sess.run (self.global_step)
            dW_train,X_train = self.sample_path(self.batch_size)  # 生成数据
            self.sess.run(self.train_op,
                          feed_dict ={self.dW : dW_train ,   # 喂数据给buildmodel中的占位符
                                      self.X : X_train ,
                                      self.is_training : True })
            if step % self.n_displaystep == 0:   # 每100步用验证集测试一下损失和Y0的值
                temp_loss = self.sess.run(self.loss ,
                                          feed_dict = feed_dict_valid)
                temp_init = self.Y0.eval()[0]    # 取出值
                self.loss_history.append(temp_loss)    # 损失值
                self.init_history.append(temp_init )   # Y0值，Y0是二维张量
                print("step : % 5u , loss : %.4e , " % \
                        ( step , temp_loss ) + \
                        " Y0 : % .4e , runtime : %4u " % \
                        (temp_init , time.time() - start_time + self.t_bd ))
            step += 1
        end_time = time.time()  # 训练结束的总时间
        print(" running time : % .3f s " % \
                ( end_time - start_time + self.t_bd ))

    def build(self):
        # build the whole network by stacking subnetworks，构架大网络
        start_time = time.time () 
        # dW、X、is_training的占位符，为什么是None,因为一次计算一个batch
        self.dW = tf.placeholder(tf.float32 ,[ None , self.d , self.n_time ] ,name = 'dW')   # None*100*20
        self.X = tf.placeholder(tf.float32 ,[ None , self.d , self.n_time +1] ,name = 'X')   # None*100*20
        self.is_training = tf.placeholder (tf.bool)
        
        # 初始化Y0\Z0
        self.Y0 = tf.Variable(tf.random_uniform([1] ,                      # u0初始化,一个维度一个值  
                                                minval = self.Yini [0] ,   # 最小值0.3  
                                                maxval = self.Yini [1] ,   # 最大值0.6
                                                dtype = tf.float32 ));
        self.Z0 = tf.Variable (tf.random_uniform ([1,self.d] ,    # u梯度的初始值，一个1*d 向量
                                                minval = -.1 ,   # 最小值
                                                maxval =.1 ,    # 最大值
                                                dtype = tf.float32 ))
        self.allones = \
             tf.ones(shape = tf.stack([ tf.shape(self.dW)[0],1]) ,   # tf.shape(self.dW)[0]=len(batch),shape=(batch,1)
                         dtype = tf.float32 )                        # 作用，批量（batch）产生初始值

        Y = self.allones * self.Y0  # 初始的Y作为输入,每一个batch都赋予相同的初始Y值，切Y是一个(batch,1)二维矩阵[[],[],..,]
        Z = tf.matmul(self.allones, self.Z0 )  # 初始的Z做为输出，作用和Y相同，但是由于Z是一个向量所以要乘积（batch,d）矩阵
        
        
        with tf.variable_scope('forward'):   # 前向
            for t in range(0,self.n_time -1):  # 前N-1个xt的网络
                    # 计算过程来源于递推公式
                    Y = Y - self.f_tf(self.t_stamp[t] ,       # 时间戳，累计的时间值
                                    self.X[:,:,t],Y,Z)* self.h   # Y.shape=(batch,1) 
                    Y = Y + tf.reduce_sum(Z * self.dW[:,:,t],1, # 矩阵点乘。
                                       keep_dims = True )      # 得到中间时刻的输出。
                    Z = self._one_time_net(self.X[:,:,t +1] ,  # 得到“u梯度”-- 由神经网络训练而来的。
                                       str(t +1))/self.d      # str(t +1) 名字，why /self.d ?
            # terminal time，因为最后一刻的Y不用神经网络了
            Y = Y - self.f_tf(self.t_stamp[self.n_time -1] ,   # 为什么-1,下标从0开始的，最后一刻有所不同。
                                  self.X[:,:,self.n_time -1] , # 
                                  Y,Z)* self.h
            Y = Y + tf.reduce_sum(Z * self.dW [:,:,self.n_time -1] , 1 ,
                                      keep_dims = True )   # 最后一次的Y,也即最终输出
            term_delta = Y - self.g_tf(self.T,
                                   self.X[:,:,self.n_time]) # 损失函数公式
            self.clipped_delta = \
                  tf.clip_by_value(term_delta ,-50.0 , 50.0)  #将一个张量中的数值限制在一个范围之内，大于为上界，小于为下界
            self.loss = tf.reduce_mean(self.clipped_delta**2)  #计算损失，来源于损失函数
        self.t_bd = time.time() - start_time  # 生成网络的时间

    def sample_path(self, n_sample):
        # 产生路径，生成（xt,(wt-wt-1)）
        dW_sample = np.zeros([n_sample,self.d,self.n_time])   # 样本数、维度、时间长度
        X_sample = np.zeros([n_sample,self.d,self.n_time +1])
        for i in range(self.n_time):    # 一次生成一列
            dW_sample [:,:,i] = \
               np.reshape(normal.rvs(mean = np.zeros(self.d) ,  # 该函数相当于np.random.normal()
                                     cov =1 ,   # 为什么不是std=1 ?
                                     size = n_sample)* self.sqrth ,   # 根号delta t，W(t)-W(s)独立于的W(r),且是期望为0方差为t-s
                          (n_sample,self.d))
            X_sample[:,:,i +1] = X_sample[:,:,i] + \
                                   np.sqrt(2)*dW_sample[:,:,i]   # 来自公式
        return dW_sample, X_sample

    def f_tf(self,t,X,Y,Z ):
        # nonlinear term
        return Y - tf.pow(Y,3)

    def g_tf(self,t,X):
        # terminal conditions
        return 0.5/(1 + 0.2* tf.reduce_sum(X **2,1,keep_dims = True ))

    def _one_time_net(self , x ,name ):
        # 一个batch在t时刻的网络构架，输出梯度
        with tf.variable_scope(name):
            x_norm = self._batch_norm(x , name = 'layer0_normal')  # 对batch标准化，作为输入
            layer1 = self._one_layer(x_norm , self.n_neuron [1] ,   # 隐藏层1输入输出input(batch,d),output(batch，d+10)
                                      name = 'layer1')
            layer2 = self._one_layer(layer1,self.n_neuron[2] ,  # 隐藏层2 input(batch,d+10),output(batch,d+10)
                                      name = 'layer2')
            z = self._one_layer(layer2 , self.n_neuron [3] , #  输出层，不加relu函数做激活input(batch,d+10),output(baatch,d)
                                     activation_fn = None , name = 'final')
        return z

    def _one_layer(self , input_ , out_sz ,
                   activation_fn = tf.nn.relu ,
                   std =5.0 , name = 'linear'):
        # 一个层里面的计算操作,一个层的输入，返回一个层的输出0，被_one_time_net调用用于构造一个时刻的神经网络
        with tf.variable_scope(name):
            shape = input_.get_shape().as_list()  # list(输入的维度)
            w = tf.get_variable('Matrix',     # get_variable它一定是和tf.variable_scope()共同使用的
                                [shape[1], out_sz] ,tf.float32,    # 输入维度和输出维度
                                norm_init(stddev = \
                                          std / np.sqrt(shape[1]+ out_sz )))  # 生成权重矩阵
            hidden = tf.matmul(input_ ,w )  # 矩阵乘积，隐藏计算中间结果
            hidden_bn = self._batch_norm(hidden, name = 'normal')  # batch标准化
        if activation_fn != None :
            return activation_fn(hidden_bn)  # 激活函数
        else :
            return hidden_bn  #不加激活函数,线性

    def _batch_norm(self , x , name ):
        """ Batch normalization """ # beta、gamma需要训练，第三类参数来源,一次标准化需要2列参数
        with tf.variable_scope(name):
            params_shape = [x.get_shape()[ -1]]   # [d,d+10,d+10,d]，第一个维度是batch
            beta = tf.get_variable('beta', params_shape ,
                                         tf.float32 ,
                                         norm_init(0.0 , stddev =0.1 ,
                                         ))
            gamma = tf.get_variable( 'gamma', params_shape ,
                                         tf.float32 ,
                                         unif_init (0.1,0.5 ,
                                          ))
            mv_mean = tf.get_variable('moving_mean' ,   # 由于每次的batch不同，所以用moving_mean来改进mean
                                         params_shape ,
                                         tf.float32 ,
                                         const_init (0.0) ,
                                         trainable = False )
            mv_var = tf.get_variable('moving_variance' ,
                                        params_shape ,
                                        tf.float32 ,
                                        const_init(1.0) ,
                                        trainable = False )
            
            # These ops will only be preformed when training
            mean ,variance = tf.nn.moments(x ,[0] , name = 'moments')#需要标准化的中心维度,[0]表示batch,求64个数据的均值方差
            self._extra_train_ops.append (\
                 assign_moving_average(mv_mean , mean , 0.99))  # 下面详解
            self._extra_train_ops.append (\
                 assign_moving_average(mv_var , variance , 0.99))
            
            mean,variance = \
                control_flow_ops.cond(self.is_training ,            # control_flow_ops.cond控制执行流，第一个为条件
                                     lambda :( mean , variance ) , # 条件True时执行，train时，需要进行重新求均值方差
                                     lambda :( mv_mean , mv_var )) # 条件False时执行,test时，直接调用最后一次的平滑值
            
            y = tf.nn.batch_normalization (x , mean , variance ,
                                           beta , gamma , 1e-6)   
            # 上面一步的操作相当于:  
            # y = (y - mean)/tf.sqrt(variance+1e-6)  # 1e-6 epslion
            # y = y * gamma + beta
            
            # 确保标准化后的形状不变
            y.set_shape( x.get_shape())
            return y

def main ():
    tf.reset_default_graph ()
    with tf.Session() as sess :
        tf.set_random_seed (1)  # tf中的随机种子
        print(" Begin to solve Allen - Cahn equation ")
        model = SolveAllenCahn (sess)  # 创建对象
        model.build()   # 调用对象方法，构建了一个模型，即定义了各个解，但没有传入数据
        model.train ()  # 生成并传数据到build
        output = np.zeros ((len(model.init_history), 3))   # 初始化结果为0,后面填充
        output[:,0] = np.arange(len( model.init_history )) \
                             * model.n_displaystep         # 输出step
        output[:,1] = model.loss_history  # 输出loss列表
        output[:,2] = model.init_history # 输出 Y0列表
        np.savetxt("./ AllenCahn_d100.csv " ,  # 保存输出结果
                     output ,
                     fmt =[ '%d', '%.5e', '%.5e'] ,
                     delimiter =",",
                     header ="step ,loss function , " + \
                     " target value , runtime " ,
                     comments = '')

if __name__ == '__main__':
        np.random.seed(1) # 定义一个随机数种子
        for i in range(5):
            print(str(i)+' run:')
            main()  # 运行主程序

0 run:
 Begin to solve Allen - Cahn equation 
step :     0 , loss : 1.6158e-01 , Y0 :  5.4381e-01 , runtime :   10 
step :  1000 , loss : 1.1286e-02 ,  Y0 :  1.9333e-01 , runtime :  109 
step :  2000 , loss : 3.8380e-04 ,  Y0 :  6.9254e-02 , runtime :  201 
step :  3000 , loss : 2.3479e-04 ,  Y0 :  5.3180e-02 , runtime :  293 
step :  4000 , loss : 1.9822e-04 ,  Y0 :  5.3185e-02 , runtime :  382 
 running time :  382.833 s 
1 run:
 Begin to solve Allen - Cahn equation 
step :     0 , loss : 1.5930e-01 , Y0 :  5.4381e-01 , runtime :   10 
step :  1000 , loss : 1.1439e-02 ,  Y0 :  1.9381e-01 , runtime :  104 
step :  2000 , loss : 4.1777e-04 ,  Y0 :  6.9284e-02 , runtime :  195 
step :  3000 , loss : 2.6188e-04 ,  Y0 :  5.3316e-02 , runtime :  287 
step :  4000 , loss : 2.2518e-04 ,  Y0 :  5.3059e-02 , runtime :  379 
 running time :  379.859 s 
2 run:
 Begin to solve Allen - Cahn equation 
step :     0 , loss : 1.6276e-01 , Y0 :  5.4381e-01 , runtime :   11 
step :  1000 , loss : 1.1409

### 四、BN
![image.png](attachment:image.png)

#### tf.nn.moments输出就是BN需要的mean和variance

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()
img = tf.random_normal([2, 3])
axis = list(range(len(img.get_shape()) - 1))
mean, variance = tf.nn.moments(img, axis)
img.eval(),mean.eval(),variance.eval()

D:\Anaconda\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(array([[ 1.2752138 , -1.3859133 ,  0.44791156],
        [-0.2962764 , -0.01701711,  0.16417223]], dtype=float32),
 array([ 1.368106  ,  0.68490934, -0.598477  ], dtype=float32),
 array([0.07034072, 1.6729448 , 0.6175715 ], dtype=float32))

#### assign_moving_average() 移动平均

assign_moving_average(variable, value, decay, zero_debias=True, name=None)<br>
其实内部计算比较简单，公式表达如下：<br>
variable = variable * decay + value * (1 - decay)